In [64]:
import numpy as np
import pandas as pd

#X = onehotencoder.fit_transform(imagesArray.reshape(-1,1)).toarray()
#y = dataSet["objeto"]
from keras.utils.np_utils import to_categorical
ames = pd.read_csv('ames.csv', nrows= 1e6)


In [65]:
np.random.seed(42)


variables_independientes = ames.drop("precio_venta", axis=1).columns
datos_numericos = ames[variables_independientes].select_dtypes([int, float, "float64","int64"])
col_no_numericas = ames[variables_independientes].select_dtypes([object]).columns
col_numericas = datos_numericos.columns

dict_var_ordinales = {
    "calidad_cocinas": ["Po", "Fa", "TA", "Gd", "Ex"],
    "funcionalidad":["Sal", "Sev", "Maj2", "Maj1", "Min2", "Min1","Typ"],
    "calidad_chimeneas":["NA","Po","Fa","TA","Gd","Ex"],
    "acabado_garaje":["NA","Unf","RFn","Fin"],
    "calidad_garaje":["NA","Po","Fa","TA","Gd","Ex"],
    "condicion_garaje":["NA","Po","Fa","TA","Gd","Ex"],
    "acceso_garaje_pavimentado":["N", "P", "Y"],
    "calidad_piscina":["NA","Fa","TA","Gd","Ex"],
    "calidad_valla":["NA","MnWw","GdWo","MnPrv","GdPrv"],
    "forma_parcela":["IR3", "IR2", "IR1","Reg"],
    "tipo_instalaciones":["ELO","NoSeWa","NoSewr","AllPub"],
    "pendiente_parcela":["Sev", "Mod", "Gtl"],
    "calidad_material_exterior":["Po","Fa","TA","Gd","Ex"],
    "condicion_material_exterior":["Po","Fa","TA","Gd","Ex"],
    "altura_sotano":["NA","Po","Fa","TA","Gd","Ex"],
    "condicion_sotano":["NA","Po","Fa","TA","Gd","Ex"],
    "sotano_exterior":["NA","No","Mn","Av","Gd"],
    "calidad_sotano_habitable1":["NA", "Unf", "LwQ", "Rec", "BLQ", "ALQ", "GLQ"],
    "calidad_sotano_habitable2":["NA", "Unf", "LwQ", "Rec", "BLQ", "ALQ", "GLQ"],
     "calidad_calefaccion":["Po","Fa","TA","Gd","Ex"],
    
}

col_ordinales = list(dict_var_ordinales.keys())
datos_ordinales = ames[col_ordinales]
col_categoricas = list(set(col_no_numericas) - set(col_ordinales))
datos_categoricos = ames[col_categoricas]



In [66]:
from sklearn.preprocessing import  Normalizer
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(missing_values=np.nan, strategy='median')
datos_imputer = imp_mean.fit_transform(datos_numericos);

transformer = Normalizer().fit(datos_imputer)  # fit does nothing.
dator_normalizados = transformer.transform(datos_imputer)
datos_numericos_imputados_normalizados = pd.DataFrame(
    dator_normalizados,
    columns=datos_numericos.columns
)

In [67]:
datos_categoricos_dummy = pd.get_dummies(datos_categoricos, drop_first=True)


In [68]:
for columna_ordinal, valores in dict_var_ordinales.items():
    datos_ordinales[columna_ordinal] = (
    datos_ordinales[columna_ordinal]
    .astype("category")
    .cat.set_categories(valores)
    .cat.codes
)
    


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [69]:

ames_procesado = pd.concat([
    datos_numericos_imputados_normalizados,
    datos_categoricos_dummy,
    datos_ordinales
], axis=1)
variable_objetivo = ["precio_venta"]
variables = ["area_parcela","n_chimeneas","n_cocinas","n_dormitorios_sobre_suelo","area_habitable_sobre_suelo", "calidad_material_exterior","calidad_chimeneas"]
ames_X = ames_procesado[variables]
ames_y = ames[variable_objetivo]


print(ames_X.dtypes)

area_parcela                  float64
n_chimeneas                   float64
n_cocinas                     float64
n_dormitorios_sobre_suelo     float64
area_habitable_sobre_suelo    float64
calidad_material_exterior        int8
calidad_chimeneas                int8
dtype: object


In [70]:



from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

X_train, X_test ,y_train,  y_test = train_test_split(ames_X, ames_y)
modelo = LinearRegression()
modelo.fit(X=X_train,
           y=y_train)

y_pred = modelo.predict(X_test)

#print(X_train, X_test)
from sklearn.metrics import mean_squared_error
mse = np.sqrt(mean_squared_error(y_test, y_pred))
print(mse)

41196.95894269321


In [71]:
from sklearn.externals import joblib
import json
joblib.dump(modelo, 'modelo_ames.pkl') 
import json
with open('columnas_names.json', 'w') as fname:
    x_columnas = ames_X.columns.tolist()
    json.dump(x_columnas, fname)

x_category = {}
x_dtypes = ames_X.dtypes
for col in ames_X.select_dtypes(['int8']):
    x_category[col] = json.dumps(ames_X[col].unique().tolist())
        
print(x_category)

x_dtypes = {col: str(ames_X[col].dtype) for col in ames_X.columns}

x_label = {col: col for col in ames_X.columns}
joblib.dump(x_dtypes, 'dtypes_ames.pkl')
joblib.dump(x_label, 'dlabel_ames.pkl')
joblib.dump(x_category, 'dcategory_ames.pkl')

{'calidad_material_exterior': '[2, 3, 4, 1]', 'calidad_chimeneas': '[4, -1, 3, 1, 5, 2]'}


['dcategory_ames.pkl']